# Train NN to generate total latent heat given only forcing data

In [ ]:
%pylab inline
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pandas as pd
from glob import glob
import xarray as xr
import seaborn as sns
from IPython.display import SVG
from tqdm.keras import TqdmCallback
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow import keras
from tensorflow.keras.utils import plot_model, model_to_dot
from tensorflow.keras import layers
import dask.dataframe as dd
from sklearn import preprocessing
from tensorflow.keras.callbacks import Callback, EarlyStopping
from KerasWeightsProcessing.convert_weights import txt_to_h5, h5_to_txt
import pysumma.plotting as psp
import pysumma.utils as psu
import sklearn

sns.set_context('talk')
mpl.style.use('seaborn-bright')
mpl.rcParams['figure.figsize'] = (18, 12)

#strategy = tf.distribute.MirroredStrategy()
#print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
dtype='float32'
K.set_floatx(dtype)

In [ ]:
sim_sites = os.listdir('../sites')

seed = 50334
np.random.seed(seed)
np.random.shuffle(sim_sites)
len(sim_sites)

In [ ]:
site_dict = {s: xr.open_dataset(f'../sites/{s}/forcings/{s}.nc').isel(hru=0, drop=True).load() 
             for s in sim_sites}

site_attr = {s: xr.open_dataset(f'../sites/{s}/params/local_attributes.nc').isel(hru=0, drop=True).load() 
             for s in sim_sites}

site_outp = {s: xr.open_dataset(f'../sites/{s}/lrp_nn_output_{s}_timestep.nc').isel(hru=0, drop=True).load() 
             for s in sim_sites}

site_parm = {s: xr.open_dataset(f'../sites/{s}/params/parameter_trial.nc').isel(hru=0, drop=True).load() 
             for s in sim_sites}

def trim_time(sim, obs, roundto='min'):
    sim['time'] = sim['time'].dt.round(roundto)
    obs['time'] = obs['time'].dt.round(roundto)
    sim_start = sim['time'].values[1]
    sim_stop = sim['time'].values[-2]
    obs_start = obs['time'].values[1]
    obs_stop = obs['time'].values[-2]
    start = max(sim_start, obs_start)
    stop = min(sim_stop, obs_stop)
    return slice(start, stop)


for s in sim_sites:
    # trim times to match
    in_ds = site_dict[s]
    outp_ds = site_outp[s]
    ts = trim_time(in_ds, outp_ds)
    in_ds = in_ds.sel(time=ts)
    outp_ds = outp_ds.sel(time=ts)
    site_dict[s] = in_ds
    site_outp[s] = outp_ds
    

In [ ]:
def calc_relhum(T, p, sh):
    T0 = 273.16
    return sh * (0.263*p)  / np.exp((17.67*(T-T0))/(T-29.65))

def etl_single_site(in_ds, attr_ds, outp_ds, parm_ds, use_mask=True):
   
    #---------------------------------------------------------------------------
    # Forcings
    #---------------------------------------------------------------------------
    airtemp   = (((in_ds['airtemp'].values / 27.315) - 10) / 2) + 0.5
    spechum   = (in_ds['spechum'].values * 50)  
    swradatm  = (in_ds['SWRadAtm'].values / 1000) 
    lwradatm  = in_ds['LWRadAtm'].values / (2 * 273.16)
    pptrate   = np.around(10 * np.cbrt(in_ds['pptrate'].values), 2)
    airpres   = (10 - (in_ds['airpres'].values / 10132.5)) / 3
    windspd   = in_ds['windspd'].values / 10
    mask      = in_ds['gap_filled'].values
    relhumid  = calc_relhum(in_ds['airtemp'].values, in_ds['airpres'].values, in_ds['spechum']) / 100
    relhumid[relhumid<0] = 0
    #---------------------------------------------------------------------------
    # Parameters
    #---------------------------------------------------------------------------
    soiltype = attr_ds['soilTypeIndex'].values[()] * np.ones_like(mask) / 12
    vegtype = attr_ds['vegTypeIndex'].values[()] * np.ones_like(mask) / 12
    lai = outp_ds['scalarLAI'].values / 12
    try:
        canwidth = (parm_ds['heightCanopyTop']).values / 20
    except:
        canwidth = 1/(lai + 0.001)
    
    thetasat = parm_ds['theta_sat'].values[()] * np.ones_like(mask)
    thetares = parm_ds['theta_res'].values[()] * np.ones_like(mask)
    fieldcapacity = parm_ds['fieldCapacity'].values[()] * np.ones_like(mask)
    soilwilting = parm_ds['critSoilWilting'].values[()] * np.ones_like(mask)
    sm_min = soilwilting
      
    #---------------------------------------------------------------------------
    # Soil moistures
    #---------------------------------------------------------------------------
    # Surface moisture
    nlayers_top = 0
    nlayers_bot = 4
    surf_idx = -len(outp_ds.midSoil)
    surf_sm = outp_ds['mLayerVolFracWat'].copy(deep=True)
    vmask = surf_sm != -9999
    
    depth = outp_ds['mLayerHeight'].copy(deep=True)
    dmask = depth != -9999
    depth.values = psp.utils.justify(depth.where(dmask).values)
    depth = depth.isel(midToto=slice(surf_idx+nlayers_top, surf_idx+nlayers_bot))
    depth = depth / depth.sum(dim='midToto')
    
    surf_sm.values = psp.utils.justify(surf_sm.where(vmask).values)
    surf_sm = surf_sm.isel(midToto=slice(surf_idx+nlayers_top, surf_idx+nlayers_bot))
    surf_sm *= depth
    surf_sm = surf_sm.sum(dim='midToto')
    surf_sm = (surf_sm - sm_min[0]) / (thetasat[0] - sm_min[0])
   
    #---------------------------------------------------------------------------
    # Arrange inputs and outputs
    #---------------------------------------------------------------------------    
    train_input = np.vstack([airtemp,       # 0
                             relhumid,      # 1
                             swradatm,      # 2
                             surf_sm,       # 3
                             lai * canwidth, # 4
                             vegtype,       # 5
                            ]).T 
    train_output = np.vstack([in_ds['Qle_cor'].values / 500,
                              in_ds['Qh_cor'].values / 500,]).T
    
    if use_mask:
        train_input = train_input[mask == 0]
        train_output = train_output[mask == 0]    
    return train_input.astype(np.float32), train_output.astype(np.float32)

In [ ]:
# -----------------------------------------------
# load in data, transform, and split for training
# -----------------------------------------------
train_data = [etl_single_site(site_dict[s], site_attr[s], site_outp[s], site_parm[s]) for s in sim_sites]

train_input = np.vstack([td[0] for td in train_data])
train_output = np.vstack([td[1] for td in train_data])

index_shuffle = np.arange(train_output.shape[0])
np.random.shuffle(index_shuffle)
train_input = train_input[index_shuffle, :]
train_output = train_output[index_shuffle, :]

validation_frac = 0.2
validation_start_idx = int(train_output.shape[0] * (1-validation_frac))

train_input, valid_input = train_input[0:validation_start_idx, :], train_input[validation_start_idx:, :]
train_output, valid_output = train_output[0:validation_start_idx], train_output[validation_start_idx:]    

assert np.isnan(train_input).sum() + np.isnan(train_output).sum() == 0

In [ ]:
# -----------------------------------------------
# Define model hyperparameters
# -----------------------------------------------
model_name = f'simple_dense_dropout'
loss = 'mse'
activation = 'tanh'    
width = 28
dropout_rate = 0.1
epochs = 50
batch_size = 48 * 28
num_layers = 2
use_dropout = True
optimizer = 'adam' 

# -----------------------------------------------
# Define model structure
# -----------------------------------------------
input_layer = layers.Input(shape=train_input[0].shape)
l = input_layer

for _ in range(num_layers):
    l = layers.Dense(width, activation=activation)(l)
    if use_dropout:
        l = layers.Dropout(dropout_rate)(l)
output_layer = layers.Dense(train_output.shape[-1], activation='linear')(l)

model = keras.Model(inputs=input_layer, outputs=output_layer)
model.compile(loss=loss, optimizer=optimizer)
print(model.summary())

In [ ]:
# -----------------------------------------------
# Train the model
# -----------------------------------------------
history = model.fit(
    train_input, train_output,
    validation_data=(valid_input, valid_output),
    batch_size=batch_size, epochs=epochs, shuffle=True, verbose=1, 
    callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

# -----------------------------------------------
# Save the model and history
# -----------------------------------------------
# save the history     
hist_df = pd.DataFrame(history.history) 
hist_csv_file = f'../new_models/{model_name}.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

# save the model    
model.save(f'../new_models/{model_name}.h5')
h5_to_txt(
    weights_file_name=f'../new_models/{model_name}.h5', 
    output_file_name=f'../new_models/{model_name}.txt'
)

In [ ]:
mpl.rcParams['figure.figsize'] = (18, 12) 
plt.plot(history.history['loss'][:], label='training', linewidth=3, color='tomato')
plt.plot(history.history['val_loss'][:], label='validation', linewidth=3, color='olivedrab')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.legend()
plt.show()